In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import json
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display
import os

In [2]:
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

In [3]:
ipath = ('/home/aa/cloudJ/JB/PRD/scrapper/Scripts/Renner/Salida/dfRenner_femenino2019-12-19.xlsx')
dfRenner_moda = pd.read_excel(ipath)

In [4]:
ipath = ('/home/aa/cloudJ/JB/PRD/scrapper/Scripts/Renner/Salida/dfRenner_deportivo2019-12-19.xlsx')
dfRenner_deportivo = pd.read_excel(ipath)

In [5]:
ipath = ('/home/aa/cloudJ/JB/PRD/scrapper/Scripts/Renner/Salida/dfRenner_moda_intima2019-12-19.xlsx')
dfRenner_moda_intima = pd.read_excel(ipath)

In [6]:
dfRenner = pd.concat([dfRenner_moda,dfRenner_deportivo,dfRenner_moda_intima],sort=True)

In [7]:
dfRenner = dfRenner.drop_duplicates()

In [8]:
dfRenner = dfRenner.reset_index(drop=True)

In [9]:
dfRenner.columns

Index(['Color', 'Descripcion', 'Fecha', 'Marca', 'Precio', 'Precio Original',
       'Tipo', 'Url Imagen', 'Url Producto', 'id Producto'],
      dtype='object')

In [10]:
dfRenner['Color'].unique()

array(['Rosa', 'Vermelho', 'Amarelo', 'Marrom', 'Bege', 'Roxo', 'Branco',
       'Verde', 'Laranja', 'Preto', 'Cinza', 'Multicores', 'Azul',
       'Prata', 'Sortidas'], dtype=object)

In [12]:
#COLOR ESPAÑOL
color_pr = list(dfRenner['Color'].unique())

color_esp =['Rosa','Rojo','Amarillo','Marrón','Beige','Morado','Blanco',
            'Verde','Naranja','Negro','Gris','Multicolor','Azul',
            'Plata','Surtido']#

color = [[pr.upper(),esp.upper()] for pr,esp in zip(color_pr,color_esp)]

In [13]:
dfColor = pd.DataFrame(color,columns=(['BR','AR']))

In [14]:
dfRenner['COLOR_AUX'] = [dfColor[dfColor['BR'] == row['Color'].upper()]
                             .iloc[0]['AR'] if row["Color"].upper() in dfColor['BR']
                             .values else 'No encontrado' for index,row in dfRenner.iterrows()]

In [17]:
dfRenner[['COLOR_AUX','Color']]

,COLOR_AUX,Color
0,ROSA,Rosa
1,ROSA,Rosa
2,ROSA,Rosa
3,ROSA,Rosa
4,ROSA,Rosa
5,ROSA,Rosa
6,ROSA,Rosa
7,ROSA,Rosa
8,ROSA,Rosa
9,ROSA,Rosa


In [23]:
dfRenner['Tipo'].unique()

23

In [19]:
#DESCRIPCION ESPAÑOL

#import concurrent.futures
#import goslate
#executor = concurrent.futures.ThreadPoolExecutor(max_workers=100)
#gs = goslate.Goslate(executor=executor)
#tipo_aux = pd.Series(gs.translate(dfRenner['Tipo'].unique(), "es"))

tipo_br = list(dfRenner['Tipo'].unique())

    
tipo_esp = ['Bermudas y shorts','Blusas y camisetas','Blusas y Sweaters',
            'Pantalones','Camisas','Abrigos y chaquetas','Chalecos','Deportivos',
            'Lenceria','Monos y jardineros','Moda playa','Pijamas',
            'Faldas','Vestidos','Body','Bragas','Corset y Camisas',
            'Sweaters','Formadores y reductores','Pijamas y Short dolls',
            'Togas y batas','Tops','Conjunto']

tipo_esp = [esp.upper() for esp in tipo_esp]

tipo = [[br.upper(),esp] for br,esp in zip(tipo_br,tipo_esp)]

In [20]:
dfTipo = pd.DataFrame(tipo,columns=(['BR','AR']))

In [21]:
dfRenner['TIPO_AUX'] = [dfTipo[dfTipo['BR']==row['Tipo'].upper()]
                             .iloc[0]['AR'] if row["Tipo"].upper() in dfTipo['BR']
                             .values else 'No encontrado' for index,row in dfRenner.iterrows()]

In [24]:
len(dfRenner.dropna()) == len(dfRenner)

True

In [25]:
dfRenner = dfRenner.dropna()

In [26]:
dfRenner = dfRenner[dfRenner['Precio']!=99999]

In [27]:
len(dfRenner)

8891

In [28]:
#dfRenner.head(1)
dfRenner = dfRenner.rename(columns={0: 'Color'})
dfRenner = dfRenner.rename(columns={1: 'Descripcion'})
dfRenner = dfRenner.rename(columns={2: 'Fecha'})
dfRenner = dfRenner.rename(columns={3: 'Marca'})
dfRenner = dfRenner.rename(columns={4: 'Precio'})
dfRenner = dfRenner.rename(columns={5: 'Precio Original'})
dfRenner = dfRenner.rename(columns={6: 'Tipo'})
dfRenner = dfRenner.rename(columns={7: 'Url Imagen'})
dfRenner = dfRenner.rename(columns={8: 'Url Producto'})
dfRenner = dfRenner.rename(columns={9: 'id Producto'})
dfRenner = dfRenner.rename(columns={10: 'Color_aux'})
dfRenner = dfRenner.rename(columns={11: 'Tipo_aux'})

dfRenner['Origen'] = 'RENNER BR'
dfRenner['Moneda'] = 'REAL R$'
dfRenner['Sexo'] = 'Mujer'

In [29]:
dfRenner

,Color,Descripcion,Fecha,Marca,Precio,Precio Original,Tipo,Url Imagen,Url Producto,id Producto,COLOR_AUX,TIPO_AUX,Origen,Moneda,Sexo
0,Rosa,Bermuda Esportivo Lisa Cós Alto,2019/12/19,Get Over,59.90,59.9,Bermudas e Shorts,//img.lojasrenner.com.br/item/549323966/small/...,//www.lojasrenner.com.br/p/bermuda-esportivo-l...,549323907,ROSA,BERMUDAS Y SHORTS,RENNER BR,REAL R$,Mujer
1,Rosa,Short Liso com Amarração Curve & Plus Size,2019/12/19,Ashua Curve e Plus Size,119.90,119.9,Bermudas e Shorts,//img.lojasrenner.com.br/item/550056906/small/...,//www.lojasrenner.com.br/p/short-liso-com-amar...,550056893,ROSA,BERMUDAS Y SHORTS,RENNER BR,REAL R$,Mujer
2,Rosa,Short Clochard com Ilhós e Cinto Curve & Plus ...,2019/12/19,Ashua Curve e Plus Size,111.92,139.9,Bermudas e Shorts,//img.lojasrenner.com.br/item/550068683/medium...,//www.lojasrenner.com.br/p/short-clochard-com-...,549959227,ROSA,BERMUDAS Y SHORTS,RENNER BR,REAL R$,Mujer
3,Rosa,Short em Sarja Neon,2019/12/19,Blue Steel,89.90,89.9,Bermudas e Shorts,//img.lojasrenner.com.br/item/549502454/small/...,//www.lojasrenner.com.br/p/short-em-sarja-neon...,549502420,ROSA,BERMUDAS Y SHORTS,RENNER BR,REAL R$,Mujer
4,Rosa,Short Espotivo com Textura,2019/12/19,Get Over,31.90,39.9,Bermudas e Shorts,//img.lojasrenner.com.br/item/548618025/small/...,//www.lojasrenner.com.br/p/short-espotivo-com-...,547380908,ROSA,BERMUDAS Y SHORTS,RENNER BR,REAL R$,Mujer
5,Rosa,Short em Linho Liso com Abotoamento,2019/12/19,Blue Steel,95.90,119.9,Bermudas e Shorts,//img.lojasrenner.com.br/item/549634916/small/...,//www.lojasrenner.com.br/p/short-em-linho-liso...,549634895,ROSA,BERMUDAS Y SHORTS,RENNER BR,REAL R$,Mujer
6,Rosa,Short Clochard com Amarração em Sarja,2019/12/19,Blue Steel,89.90,99.9,Bermudas e Shorts,//img.lojasrenner.com.br/item/549471176/small/...,//www.lojasrenner.com.br/p/short-clochard-com-...,549175406,ROSA,BERMUDAS Y SHORTS,RENNER BR,REAL R$,Mujer
7,Rosa,Short Liso em Sarja,2019/12/19,Blue Steel,59.90,59.9,Bermudas e Shorts,//img.lojasrenner.com.br/item/549140262/small/...,//www.lojasrenner.com.br/p/short-liso-em-sarja...,549140246,ROSA,BERMUDAS Y SHORTS,RENNER BR,REAL R$,Mujer
8,Rosa,Short Com Babado Estampado,2019/12/19,Blue Steel,59.90,89.9,Bermudas e Shorts,//img.lojasrenner.com.br/item/548693480/small/...,//www.lojasrenner.com.br/p/short-com-babado-es...,548693463,ROSA,BERMUDAS Y SHORTS,RENNER BR,REAL R$,Mujer
9,Rosa,Short Cintura Alta com Zíper Lateral,2019/12/19,Just Be,99.90,99.9,Bermudas e Shorts,//img.lojasrenner.com.br/item/547135814/small/...,//www.lojasrenner.com.br/p/short-cintura-alta-...,547135806,ROSA,BERMUDAS Y SHORTS,RENNER BR,REAL R$,Mujer


In [30]:
len(dfRenner)

8891

{datetime.date(2019, 12, 20)}

In [36]:
writer = ExcelWriter(f'./Salida/dfRennerFULL{fecha}.xlsx')
dfRenner.to_excel(writer,'Hoja1')
writer.save()

In [37]:
ipath = (f'/home/aa/cloudJ/JB/PRD/scrapper/Scripts/Renner/Salida/dfRennerFULL{fecha}.xlsx')
dfRenner = pd.read_excel(ipath)

In [38]:
len(dfRenner)

8891

In [39]:
server = 'tcp:192.168.1.6'
database = 'planeamiento'
username = 'sa'
password = 'sa'
cnxn = pyodbc.connect('DRIVER={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.2.so.0.1};SERVER='+server+
                      ';DATABASE='+database+
                      ';UID='+username+
                      ';PWD='+ password)

In [40]:
maxCorrida = "select max(id_sc3_corrida)+1 from sc3_detalle with(nolock) where origen = 'RENNER BR' "
maxCorrida = pd.read_sql(maxCorrida,cnxn) 
maxCorrida = maxCorrida[""][0]

In [41]:
maxCorrida

10

In [42]:
cursor = cnxn.cursor()
for index,row in dfRenner.iterrows():                                                            
    cursor.execute("INSERT INTO sc3_detalle([id_sc3_producto],[id_sc3_corrida],[marca],[tipo],[tipo_es],[color],[color_es],[sexo],[descripcion],[moneda],[precio],[precio_original],[fecha_alta],[url_imagen],[url_producto],[origen]) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", 
                   row['id Producto'], 
                   int(maxCorrida),
                   row['Marca'],
                   row['Tipo'],
                   row['TIPO_AUX'],
                   row['Color'],
                   row['COLOR_AUX'],
                   row['Sexo'],
                   row['Descripcion'], 
                   row['Moneda'],
                   row['Precio'], 
                   row['Precio Original'], 
                   row['Fecha'],
                   row['Url Imagen'], 
                   row['Url Producto'],
                   row['Origen']) 
    cnxn.commit()
cursor.close()
cnxn.close()

In [43]:
len(dfRenner.dropna())

8891